
# Colorado Motor Vehicle Sales Analysis (Google Colab)
**Goal:** trends, seasonality, forecasting, and basic ML demo.


In [ ]:

#@title 🔧 Setup
import sys, os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path

# Stats / ML
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Plot config (matplotlib only; no seaborn)
plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['axes.grid'] = True

RESULTS_DIR = Path('results/plots')
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print('Versions:')
print('pandas', pd.__version__)
print('statsmodels', sm.__version__)


## 📥 Load Data

In [ ]:

#@title Choose how to load data
use_upload = False #@param {type:"boolean"}
local_path = "data/colorado_motor_vehicle_sales.csv" #@param {type:"string"}

df = None
if use_upload:
    # Upload via Colab UI
    try:
        from google.colab import files
        uploaded = files.upload()
        fname = list(uploaded.keys())[0]
        df = pd.read_csv(fname)
    except Exception as e:
        raise RuntimeError("Upload failed. Set use_upload=False to use local path.") from e
else:
    # Use local path (e.g., after cloning or in this repo structure)
    if not os.path.exists(local_path):
        # fallback: try Colab working directory if user uploaded there
        fallback = "colorado_motor_vehicle_sales.csv"
        if os.path.exists(fallback):
            local_path = fallback
        else:
            raise FileNotFoundError(f"CSV not found at {local_path}. Upload file or mount Drive.")
    df = pd.read_csv(local_path)

print(df.head())
print(df.dtypes)


## 🧹 Data Preparation

In [ ]:

# Validate expected columns
required_cols = ['year','quarter','county','sales']
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns: {missing}")

# Drop obvious duplicates
before = len(df)
df = df.drop_duplicates()
after = len(df)
print(f"Removed {before - after} duplicate rows.")

# Ensure dtypes
df['year'] = df['year'].astype(int)
df['quarter'] = df['quarter'].astype(int)
df['county'] = df['county'].astype(str)
df['sales'] = pd.to_numeric(df['sales'], errors='coerce')

# Handle missing sales
na_before = df['sales'].isna().sum()
df = df.dropna(subset=['sales'])
print(f"Dropped {na_before} rows with missing sales.")

# Create a Period label and an ordinal time index
df['period'] = df['year'].astype(str) + ' Q' + df['quarter'].astype(str)
df['t'] = (df['year'] - df['year'].min())*4 + (df['quarter'] - 1)

print(df.describe(include='all'))


## 🔎 Exploratory Data Analysis

In [ ]:

# Aggregate by year-quarter across all counties
agg = df.groupby(['year','quarter'], as_index=False)['sales'].sum()
agg['period'] = agg['year'].astype(str) + ' Q' + agg['quarter'].astype(str)

plt.figure()
plt.plot(agg['period'], agg['sales'])
plt.title('Total Colorado Motor Vehicle Sales by Quarter')
plt.xticks(rotation=90)
plt.xlabel('Period')
plt.ylabel('Sales')
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'total_sales_by_quarter.png', dpi=150)
plt.show()

# Top counties by total sales
county_totals = df.groupby('county', as_index=False)['sales'].sum().sort_values('sales', ascending=False).head(15)
plt.figure()
plt.bar(county_totals['county'], county_totals['sales'])
plt.title('Top 15 Counties by Total Sales')
plt.xticks(rotation=90)
plt.xlabel('County')
plt.ylabel('Total Sales')
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'top_counties.png', dpi=150)
plt.show()


## 📈 Seasonality (Decomposition)

In [ ]:

# Create a proper quarterly time index
# Assume data spans continuous years with quarters 1..4
ts = agg.copy()
# Build a datetime index at quarter end
ts['qdate'] = pd.PeriodIndex(year=ts['year'], quarter=ts['quarter']).to_timestamp(how='end')
ts = ts.set_index('qdate').sort_index()

# Seasonal decomposition (period=4 for quarters)
res = seasonal_decompose(ts['sales'], model='multiplicative', period=4, extrapolate_trend='freq')
fig = res.plot()
fig.set_size_inches(10, 8)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'seasonal_decomposition.png', dpi=150)
plt.show()


## 🔮 Forecasting (ARIMA demo)

In [ ]:

# Train-test split: last 4 quarters as test
train = ts.iloc[:-4]['sales']
test = ts.iloc[-4:]['sales']

# Simple ARIMA; order can be tuned
model = ARIMA(train, order=(1,1,1))
fitted = model.fit()
print(fitted.summary())

# Forecast next len(test) points
pred = fitted.forecast(steps=len(test))
rmse = mean_squared_error(test, pred, squared=False)
mae = mean_absolute_error(test, pred)
print(f"ARIMA RMSE: {rmse:.2f} | MAE: {mae:.2f}")

# Plot
plt.figure()
plt.plot(train.index, train.values, label='Train')
plt.plot(test.index, test.values, label='Test')
plt.plot(test.index, pred.values, label='Forecast')
plt.title('ARIMA Forecast vs Actuals')
plt.xlabel('Quarter')
plt.ylabel('Sales')
plt.legend()
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'arima_forecast.png', dpi=150)
plt.show()


## 🤖 ML Baseline (RandomForest — demo)

In [ ]:

# Encode county
df_ml = df.copy()
df_ml['county_code'] = df_ml['county'].astype('category').cat.codes

X = df_ml[['year','quarter','county_code']]
y = df_ml['sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
preds = rf.predict(X_test)

rmse = mean_squared_error(y_test, preds, squared=False)
mae = mean_absolute_error(y_test, preds)
print(f"RandomForest RMSE: {rmse:.2f} | MAE: {mae:.2f}")


## 🗺️ County Drilldown

In [ ]:

# Plot a single county over time
county_name = "Denver" #@param {type:"string"}
c = df[df['county'].str.lower() == county_name.lower()].groupby(['year','quarter'], as_index=False)['sales'].sum()
c['period'] = c['year'].astype(str) + ' Q' + c['quarter'].astype(str)

if len(c):
    plt.figure()
    plt.plot(c['period'], c['sales'])
    plt.title(f'{county_name} Sales by Quarter')
    plt.xticks(rotation=90)
    plt.xlabel('Period')
    plt.ylabel('Sales')
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / f'{county_name}_sales_by_quarter.png', dpi=150)
    plt.show()
else:
    print(f"No rows found for county: {county_name}")



## 📝 Findings & Takeaways (Fill after running)
- **Trend:** *(Describe overall trend)*
- **Seasonality:** *(Quarterly pattern observations)*
- **Top Counties:** *(Which ones dominate?)*
- **Forecast:** *(High-level interpretation of ARIMA results)*
- **Actions:** *(How a dealer/policy-maker might use this)*
